In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM , Bidirectional
from tensorflow.keras.layers import Dense
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from gensim.models import Word2Vec, KeyedVectors
from sklearn.model_selection import train_test_split
import re
from tensorflow.keras.layers import Dropout
import joblib

In [2]:
# Download necessary resources
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
df=pd.read_csv("train.csv")
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
num_columns = len(df.columns)
print(num_columns)

5


In [5]:
df.shape

(20800, 5)

In [6]:
df[df.duplicated()]

,id,title,author,text,label


In [7]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
df.dropna(inplace=True)

In [9]:
X=df.drop('label',axis=1)

In [10]:
y=df['label']

In [11]:
X.shape

(18285, 4)

In [12]:
y.shape

(18285,)

In [13]:
voc_size=5000

In [14]:
messages=X.copy()
messages.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [15]:
messages.reset_index(inplace=True)

### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

corpus

In [16]:
# Stopwords and Lemmatizer setup
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def get_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN 

def extract_words_with_pos(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()       
    if not text:
        return [], ""  # Return empty tuple if text is blank              
    word_tokens = word_tokenize(text)       
    pos_tags = pos_tag(word_tokens)         
    
    words_with_pos = [(word, tag) for word, tag in pos_tags] 
    

    lemmatized_text = [
        lemmatizer.lemmatize(word, get_pos(tag))
        for word, tag in pos_tags
    ]
    
    return words_with_pos, ' '.join(lemmatized_text) if lemmatized_text else ""

words_with_pos_list = []
corpus = []

for review in messages['title']:
    words_with_pos, processed_review = extract_words_with_pos(review)
    words_with_pos_list.append(words_with_pos)
    corpus.append(processed_review)

corpus

['house dem aide we didnt even see comeys letter until jason chaffetz tweet it',
 'flynn hillary clinton big woman on campus breitbart',
 'why the truth might get you fire',
 'civilian kill in single u airstrike have be identify',
 'iranian woman jail for fictional unpublished story about woman stone to death for adultery',
 'jackie mason hollywood would love trump if he bomb north korea over lack of trans bathroom exclusive video breitbart',
 'benot hamon win french socialist partys presidential nomination the new york time',
 'a backchannel plan for ukraine and russia courtesy of trump associate the new york time',
 'obamas organize for action partner with soroslinked indivisible to disrupt trump agenda',
 'bbc comedy sketch real housewife of isi cause outrage',
 'russian researcher discover secret nazi military base treasure hunter in the arctic photo',
 'u official see no link between trump and russia',
 're yes there be paid government troll on social medium blog forum and website

In [17]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[4487,
  3342,
  2796,
  440,
  1012,
  3301,
  1635,
  4272,
  4622,
  3195,
  1781,
  4589,
  2727,
  4279],
 [2328, 2523, 4647, 3636, 4624, 3524, 1829, 4398],
 [2975, 3995, 2242, 405, 4564, 100, 474],
 [4382, 3823, 4964, 3091, 1233, 2787, 1326, 4178, 4992],
 [487, 4624, 2154, 2002, 866, 1120, 696, 751, 4624, 4862, 4709, 373, 2002, 7],
 [3457,
  327,
  4041,
  2306,
  3225,
  3173,
  2238,
  2400,
  1091,
  1013,
  2949,
  4610,
  3678,
  4924,
  1846,
  158,
  3308,
  1167,
  4398],
 [2522, 1489, 1467, 2662, 4744, 2062, 3243, 2201, 3995, 2143, 915, 3272],
 [4282,
  2882,
  2373,
  2002,
  1536,
  3323,
  2040,
  3445,
  4924,
  3173,
  218,
  3995,
  2143,
  915,
  3272],
 [559, 3241, 2002, 1492, 3714, 1260, 2135, 2651, 4709, 4718, 3173, 4718],
 [4794, 209, 4145, 3553, 2527, 4924, 3297, 4838, 728],
 [3541, 4571, 4762, 3410, 1155, 2822, 4896, 522, 2482, 4964, 3995, 1301, 574],
 [1233, 4899, 1635, 1162, 837, 953, 3173, 3323, 2040],
 [280,
  2317,
  149,
  4178,
  1304,
  4343,
  4790

In [18]:
sent_length=20

In [19]:
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ..., 4589, 2727, 4279],
       [   0,    0,    0, ..., 3524, 1829, 4398],
       [   0,    0,    0, ..., 4564,  100,  474],
       ...,
       [   0,    0,    0, ..., 2143,  915, 3272],
       [   0,    0,    0, ..., 2992, 4964,  249],
       [   0,    0,    0, ..., 3995,   70, 1170]])

In [20]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0, 4487, 3342, 2796,  440, 1012,
       3301, 1635, 4272, 4622, 3195, 1781, 4589, 2727, 4279])

In [21]:
embedded_vector_features=80

In [22]:
model=Sequential()

In [23]:
model.add(Embedding(voc_size,embedded_vector_features,input_length=sent_length,trainable=True))
model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(50)))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))

c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [24]:
model.build((None, sent_length))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 20, 80)         │       400,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 20, 200)        │       144,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 20, 200)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 100)            │       100,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 645,301 (2.46 MB)

 Trainable params: 645,301 (2.46 MB)

 Non-trainable params: 0 (0.00 B)

None


In [25]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [26]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [27]:
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.33,random_state=42)

In [28]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20,batch_size=64)

Epoch 1/20


192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 37ms/step - accuracy: 0.8044 - loss: 0.3688 - val_accuracy: 0.9138 - val_loss: 0.1955
Epoch 2/20
192/192 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 0.9546 - loss: 0.1205 - val_accuracy: 0.9175 - val_loss: 0.2142
Epoch 3/20
192/192 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 0.9751 - loss: 0.0722 - val_accuracy: 0.9100 - val_loss: 0.2452
Epoch 4/20
192/192 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 0.9852 - loss: 0.0492 - val_accuracy: 0.9163 - val_loss: 0.2598
Epoch 5/20
192/192 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 0.9917 - loss: 0.0265 - val_accuracy: 0.9167 - val_loss: 0.3434
Epoch 6/20
192/192 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - accuracy: 0.9961 - loss: 0.0134 - val_accuracy: 0.9168 - val_loss: 0.3968
Epoch 7/20
192/192 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.9970 - loss: 0.0119 - val_accuracy: 0.9128 - val_loss: 0.4358
Epoch 8/20
192/192 ━━━━━━━━━━━━━━━━━━━━ 6s 34ms/step - accuracy: 0.9969 - loss: 0.0124 - val_accuracy: 0.91

In [29]:
y_pred=model.predict(X_test)
y_pred=np.where(y_pred>0.6 , 1, 0)

189/189 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step


In [30]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9161557580778791

In [31]:
joblib.dump(model,'fake_news_detetion_model.pkl')

['fake_news_detetion_model.pkl']